In [19]:
import pandas as pd




In [75]:
races=pd.read_csv('races.csv')

races=races[races['year']>=2009]

## save last race to find constructors and drivers champion


last_race_by_season=races.groupby('year')['raceId'].max()

races=races[races['year']>=2010]

races=races[['raceId','date','year','round','circuitId']]


circuits=pd.read_csv('circuits.csv')


drivers=pd.read_csv('drivers.csv')


lap_times=pd.read_csv('lap_times.csv')


lap_markers=lap_times[['lap','raceId']].groupby(['raceId'],as_index=False).max()

import math

lap_markers['lap_marker_1']=(lap_markers['lap']*.25).astype(int)
lap_markers['lap_marker_2']=(lap_markers['lap']*.5).astype(int)


lap_times['race_leader_marker_1']=lap_times['driverId']
lap_times['race_leader_marker_2']=lap_times['driverId']

lap_markers=lap_markers.merge(lap_times[['race_leader_marker_1','lap','raceId']][lap_times['position']==1],how='inner',left_on=['lap_marker_1','raceId'],right_on=['lap','raceId'])

lap_markers=lap_markers.merge(lap_times[['race_leader_marker_2','lap','raceId']][lap_times['position']==1],how='inner',left_on=['lap_marker_2','raceId'],right_on=['lap','raceId'])


lap_markers=lap_markers[['raceId','race_leader_marker_1','race_leader_marker_2']]




pit_stops=pd.read_csv('pit_stops.csv')



pit_stops['milliseconds']=pit_stops['milliseconds'].astype(float)

pit_stops=pit_stops[['raceId','driverId','milliseconds']].groupby(['raceId','driverId'],as_index=False).mean('milliseconds')

pit_stops['avg_pit_stop_seconds']=pit_stops['milliseconds']/1000


print(pit_stops)


def belongsTo(value,target):
    if value==target:
        return 1
    else: 
        return 0

circuits['isAmericas']=circuits['continent'].apply(lambda x:belongsTo(x,'Americas'))
circuits['isAustralia']=circuits['continent'].apply(lambda x:belongsTo(x,'Australia'))
circuits['isEurope']=circuits['continent'].apply(lambda x:belongsTo(x,'Europe'))


results=pd.read_csv('results.csv')


results['position']=results['position'].replace('\\N',None)

results['position']=results['position'].astype(float)

weather=pd.read_csv('visual_crossing_weather_data.csv')




def rainfall(rainfall):
    if rainfall>0:
        return 1 
    else:
        return 0



weather=weather[['raceId','temp','pressure','humidity','precip','windspeed']]



weather=weather.groupby(['raceId'],as_index=False).mean()


weather['hadRain']=weather['precip'].apply(lambda x: rainfall(x))
weather['avg_temp']=weather['temp']
weather['avg_humidity']=weather['humidity']
weather['avg_pressure']=weather['pressure']








weather['avg_wind_speed']=weather['windspeed']


weather=weather[['raceId','hadRain','avg_temp','avg_humidity','avg_pressure','avg_wind_speed']]



constructor_standings=pd.read_csv('constructor_standings.csv')


driver_standings=pd.read_csv('driver_standings.csv')






last_race_by_season=last_race_by_season.to_frame('raceId')

last_race_by_season['championship_year']=last_race_by_season.index

last_race_by_season['year']=last_race_by_season['championship_year']+1


last_race_by_season=last_race_by_season.merge(constructor_standings[['raceId','constructorId']][constructor_standings['position']==1],on=['raceId'])

last_race_by_season=last_race_by_season.merge(driver_standings[['raceId','driverId']][driver_standings['position']==1],on=['raceId'])

last_race_by_season['is_reigning_constructor_champion']=1
last_race_by_season['is_reigning_driver_champion']=1

driver_standings['raceId']=driver_standings['raceId']+1
constructor_standings['raceId']=constructor_standings['raceId']+1


driver_standings['driverPositionPreRace']=driver_standings['position']
constructor_standings['constructorPositionPreRace']=constructor_standings['position']



qualifying=pd.read_csv('qualifying.csv')


qualifying['q1'].fillna("\\N", inplace = True)
qualifying['q2'].fillna("\\N", inplace = True)
qualifying['q3'].fillna("\\N", inplace = True)


def convertLapTime(lap_time_string):

    if lap_time_string=='\\N' or lap_time_string is None:
        return None
    else:
        split=lap_time_string.split(':',1)
        minute=split[0]
        seconds=split[1]
    
        minute=int(minute)*60
        seconds=float(seconds)
    
        return minute+seconds

for a_col in ['q1','q2','q3']:
    qualifying[a_col]=qualifying[a_col].apply(lambda x: convertLapTime(x))





      raceId  driverId  milliseconds  avg_pit_stop_seconds
0        841         1       23213.0               23.2130
1        841         2       24046.0               24.0460
2        841         3       23716.0               23.7160
3        841         4       24055.0               24.0550
4        841         5       24865.0               24.8650
...      ...       ...           ...                   ...
5341    1132       848       32076.0               32.0760
5342    1132       852       29829.0               29.8290
5343    1132       855       30306.0               30.3060
5344    1132       857       29370.5               29.3705
5345    1132       858       31122.5               31.1225

[5346 rows x 4 columns]


C:\Users\melli\AppData\Local\Temp\ipykernel_13584\3959512672.py:152: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  qualifying['q1'].fillna("\\N", inplace = True)
C:\Users\melli\AppData\Local\Temp\ipykernel_13584\3959512672.py:153: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

In [45]:
# lap_times=lap_times.merge(races[['raceId','circuitId']],on=['raceId'])

# lap_times=lap_times.merge(circuits[['circuitId','name']],on=['circuitId'])










In [76]:

qualifying=qualifying[qualifying['position']<=5]

qualifying['q3_lap_time_difference'] = (qualifying.groupby('raceId')['q3']
                                                  .diff()
                                                  .fillna(0, downcast='int'))

qualifying['q1_rank']=(qualifying.groupby('raceId')['q1']
                                                  .rank()
                                                  .fillna(0, downcast='int'))

qualifying['q2_rank']=(qualifying.groupby('raceId')['q2']
                                                  .rank()
                                                  .fillna(0, downcast='int'))


def finishedFirst(rank):
    if rank==1.0:
        return 1
    else:
        return 0

qualifying['finished_first_q1']=qualifying['q1_rank'].apply(lambda x: finishedFirst(x))
qualifying['finished_first_q2']=qualifying['q2_rank'].apply(lambda x: finishedFirst(x))


lap_time_difference=qualifying[['raceId','q3_lap_time_difference','position']][qualifying['position']!=1].pivot_table(index='raceId', columns='position',values='q3_lap_time_difference', aggfunc='sum')

lap_time_difference.columns=['dif_seconds_quali_first_second','dif_seconds_quali_second_third','dif_seconds_quali_third_fourth','dif_seconds_qualif_fourth_fifth']

lap_time_difference['raceId']=lap_time_difference.index

lap_time_difference=lap_time_difference.reset_index(drop=True)



results=results.merge(qualifying[['raceId','driverId','constructorId','finished_first_q1','finished_first_q2']],
                on=['raceId','driverId','constructorId'])


results=results.merge(lap_time_difference,on=['raceId'])

results=results.merge(races[['raceId','year','round','date','circuitId']],on=['raceId'])

results=results.merge(circuits[['circuitId','isAmericas','isAustralia','isEurope']],on=['circuitId'])

results=results.merge(last_race_by_season[['year','is_reigning_constructor_champion','is_reigning_driver_champion','driverId','constructorId']],on=['year','driverId','constructorId'],how='left')

results=results.merge(drivers[['driverId','dob']],on=['driverId'])




results['date']=pd.to_datetime(results['date'])

results['dob']=pd.to_datetime(results['dob'])
results['driverAge']=(results['date']-results['dob']).dt.days


results=results.merge(driver_standings[['raceId','driverId','driverPositionPreRace']],on=['raceId','driverId'],how='left')
results=results.merge(constructor_standings[['raceId','constructorId','constructorPositionPreRace']],on=['raceId','constructorId'],how='left')


# results=results.merge(pit_stops[['raceId','driverId','avg_pit_stop_seconds']],on=['raceId','driverId'],how='left')

results=results.merge(lap_markers[['raceId','race_leader_marker_1','race_leader_marker_2']],on=['raceId'],how='left')


results['driverPositionPreRace']=results['driverPositionPreRace'].fillna(1)
results['constructorPositionPreRace']=results['constructorPositionPreRace'].fillna(1)



results['is_reigning_constructor_champion'].fillna(0,inplace=True)
results['is_reigning_driver_champion'].fillna(0,inplace=True)


def isEqual(col1_val,col2_val):
    if col1_val==col2_val:
        return 1
    else: 
        return 0

results['finishedFirst']=results.apply(lambda x: isEqual(x['grid'],x['position']),axis=1)


results['is_leader_25_percent_through']=results.apply(lambda x: isEqual(x['driverId'],x['race_leader_marker_1']),axis=1)

results['is_leader_50_percent_through']=results.apply(lambda x: isEqual(x['driverId'],x['race_leader_marker_2']),axis=1)

results['is_leader_25_percent_through'].fillna(0,inplace=True)
results['is_leader_50_percent_through'].fillna(0,inplace=True)




results=results.merge(weather,on=['raceId'])




C:\Users\melli\AppData\Local\Temp\ipykernel_13584\3598802858.py:5: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  .fillna(0, downcast='int'))
C:\Users\melli\AppData\Local\Temp\ipykernel_13584\3598802858.py:9: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  .fillna(0, downcast='int'))
C:\Users\melli\AppData\Local\Temp\ipykernel_13584\3598802858.py:13: FutureWarning: The 'downcast' keyword in fillna is deprecated and will be removed in a future version. Use res.infer_objects(copy=False) to infer non-object dtype, or pd.to_numeric with the 'downcast' keyword to downcast numeric results.
  .fillna(0, downcast='i

In [58]:
results.columns

Index(['resultId', 'raceId', 'driverId', 'constructorId', 'number', 'grid',
       'position', 'positionText', 'positionOrder', 'points', 'laps', 'time',
       'milliseconds', 'fastestLap', 'rank', 'fastestLapTime',
       'fastestLapSpeed', 'statusId', 'finished_first_q1', 'finished_first_q2',
       'dif_seconds_quali_first_second', 'dif_seconds_quali_second_third',
       'dif_seconds_quali_third_fourth', 'dif_seconds_qualif_fourth_fifth',
       'year', 'round', 'date', 'circuitId', 'isAmericas', 'isAustralia',
       'isEurope', 'is_reigning_constructor_champion',
       'is_reigning_driver_champion', 'dob', 'driverAge',
       'driverPositionPreRace', 'constructorPositionPreRace',
       'avg_pit_stop_seconds', 'race_leader_marker_1', 'race_leader_marker_2',
       'finishedFirst', 'is_leader_25_percent_through',
       'is_leader_50_percent_through', 'hadRain', 'avg_temp', 'avg_humidity',
       'avg_pressure', 'avg_wind_speed'],
      dtype='object')

In [77]:
results=results[results['grid']==1]

In [79]:
## only export columns that are actual features
results[['finished_first_q1','finished_first_q2','dif_seconds_quali_first_second','dif_seconds_quali_second_third',
'dif_seconds_quali_third_fourth','dif_seconds_qualif_fourth_fifth','isAmericas','isAustralia','isEurope','is_reigning_constructor_champion',
'is_reigning_driver_champion','driverAge','hadRain','avg_temp','avg_humidity','avg_pressure','avg_wind_speed','constructorPositionPreRace',
'driverPositionPreRace','is_leader_25_percent_through','is_leader_50_percent_through','finishedFirst']].to_csv('cleaned.csv',index=False)

,resultId,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,...,race_leader_marker_1,race_leader_marker_2,finishedFirst,is_leader_25_percent_through,is_leader_50_percent_through,hadRain,avg_temp,avg_humidity,avg_pressure,avg_wind_speed
3,20326,337,20,9,5,1,4.0,4,4,12.0,...,20.0,20.0,0,1,1,0,87.916667,35.291667,1012.550000,7.833333
9,20363,338,20,9,5,1,NaN,R,17,0.0,...,20.0,18.0,0,1,0,0,75.400000,49.168333,1009.616667,13.166667
11,20372,339,17,9,6,1,2.0,2,2,18.0,...,20.0,20.0,0,0,0,0,88.100000,67.860000,1007.600000,8.783333
18,20400,340,20,9,5,1,6.0,6,6,8.0,...,3.0,18.0,0,0,0,0,67.350000,73.275000,1016.266667,15.633333
20,20419,341,17,9,6,1,1.0,1,1,25.0,...,17.0,17.0,1,1,1,0,66.350000,61.245000,1006.733333,8.600000
